# creating-ai-influencer-Ahana

## testing

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
import PIL.Image
img = PIL.Image.open("ph.jpg")

In [ ]:
model = genai.GenerativeModel("gemini-1.0-pro-vision-latest")

In [ ]:
safety_settings_block_none={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

In [ ]:
# prompt = """ Describe the image in detail in about 50 words strictly, focusing on the model's pose, clothing, and the overall setting.
# Describe the model's pose in detail, highlighting how their body language conveys confidence or allure.
# What elements of their clothing (cut, style, color) accentuate their figure in a captivating way.
# Describe the model's posture and facial expression.
# Do they exude confidence and energy? How does their body language communicate their personality?
# Describe the model's clothing in detail.  Is it a sleek, form-fitting outfit?
# A flowing, elegant dress?  How does the clothing contribute to the overall style and mood of the image? """

In [ ]:
tp = """ Generate a detailed context from the image in strictly less than 60 words.
Focus on the model's pose, body language, and clothing.
Highlight how the model's posture and facial expression convey confidence and allure.
Describe the key elements of the model's outfit,
emphasizing how it enhances their figure and contributes to the overall stylish and modern atmosphere of the image. """

In [ ]:
response = model.generate_content([tp, img],safety_settings=safety_settings_block_none)
to_markdown(response.text)

>  The model is dressed in a black bodysuit with a plunging neckline and a white waistband, which accentuates her curves and creates a striking visual contrast. The outfit is simple yet chic, allowing the model's natural beauty to shine through. Her hair is styled in a sleek bun, and her makeup is flawless, with a smoky eye and a glossy nude lip. The model's pose is confident and alluring, with one hand on her hip and the other gently caressing her hair. Her body language exudes confidence and sensuality, and her facial expression is one of calm assurance. The overall effect is one of sophistication and elegance, with the model's outfit, pose, and facial expression all working together to create a truly stunning image.

In [ ]:
prompt_sdxl = response.text

In [ ]:
# prompt_sdxl = """The model is standing with her left hand raised, her right hand on her hip, and her head tilted slightly to the right.
# She is wearing a black bikini top and a long, black see-through skirt with a slit.
# The outfit accentuates her curves and creates a captivating silhouette.
# Her hair is long, dark, and flowing, and her makeup is flawless.
# She is looking at the camera with a confident and alluring expression.
# The overall setting is a warm and sunny day, with the light filtering through a sheer curtain behind her.
# The model's pose and facial expression exude confidence and energy, and her body language communicates her personality.
# She is a powerful and confident woman who knows what she wants and is not afraid to go after it."""

In [ ]:
neg_prompt = """ Avoid nudity, explicit sexual content, extreme poses, vulgar expressions,
inappropriate gestures, overly suggestive clothing, and any form of explicit adult content. """

In [ ]:
image = open("ph.jpg", "rb")

## finetuning-sdxl

### preparing-data

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import PIL
import pandas as pd
import os
import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel("gemini-1.0-pro-vision-latest")
safety_settings_block_none={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

In [ ]:
context_prompt = """ Generate a detailed context from the image in strictly less than 60 words.
Focus on the model's pose, body language, and clothing.
Highlight how the model's posture and facial expression convey confidence and allure.
Describe the key elements of the model's outfit,
emphasizing how it enhances their figure and contributes to the overall stylish and modern atmosphere of the image. """

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
directory = "photos"
all_photos = os.listdir(directory)
photos_paths = [f"{directory}/{path}" for path in all_photos]

In [ ]:
def get_caption(path, name):
    try:
        photo = PIL.Image.open(path)
        response = model.generate_content([context_prompt, photo],safety_settings=safety_settings_block_none)
        photo.save(f"train/{name}.jpg")
        return_context = f"a photo of ahana {response.text}"
        return return_context
    except:
        print(f"Error in getting caption of {path} ")

In [ ]:
df = pd.DataFrame(columns = ["caption", "image_file"])

In [ ]:
var = 100
for path in photos_paths:
    new_row_data = {
        "caption":get_caption(path, var),
        "image_file":f"{var}.jpg",
    }

    df.loc[len(df)] = new_row_data
    var += 1

In [ ]:
df.head()

In [ ]:
df.to_csv('train/caption.csv', index=False)

In [ ]:
import shutil
from google.colab import files

folder_to_download = "train"
output_filename = "train.zip"
shutil.make_archive("train", "zip", folder_to_download)
files.download(output_filename)

### training

In [ ]:
!pip install replicate

In [ ]:
import replicate
from google.colab import userdata
replicate_client = replicate.Client(api_token = userdata.get("REPLICATE_API_TOKEN"))

In [ ]:
training = replicate_client.trainings.create(
    version="stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
    input={
        "input_images": "https://ai-model-ahana.s3.ap-south-1.amazonaws.com/final.zip",
        "resolution": 768,
        "train_batch_size": 10,
        "num_train_epochs": 500,
        "max_train_steps": 2500,
        "is_lora": True,
        "unet_learning_rate": 1e-5,
        "ti_lr": 3e-4,
        "lora_lr": 2e-4,
        "lr_scheduler": "linear",
        "lr_warmup_steps": 100,
        "token_string": "ahana",
        "caption_prefix": "a photo of ahana",
        "mask_target_prompts": None,
        "crop_based_on_salience": True,
        "use_face_detection_instead": False,
        "clipseg_temperature": 0.9,
        "verbose": True,
        "checkpointing_steps": 200,

    },
    destination="code8soumya/ai-model-ahana"
)

In [ ]:
training.reload()
print(training.status)
print("\n".join(training.logs.split("\n")[-10:]))

processing
# PTI :step: 189, epoch: 37:   8%|▊         | 190/2500 [02:36<31:15,  1.23it/s]



# sdxl

In [35]:
!pip install -q -U google-generativeai
!pip install replicate
import replicate
import pathlib
import textwrap
import google.generativeai as genai
import PIL.Image
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [64]:
model = genai.GenerativeModel("gemini-1.0-pro-vision-latest")

In [65]:
replicate_client = replicate.Client(api_token = userdata.get("REPLICATE_API_TOKEN"))

In [138]:
tp = "a broad description about the model's body, face, cloths, background , colours "

In [139]:
safety_settings_block_none={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

In [140]:
img = PIL.Image.open("ph.jpg")

In [141]:
# img

In [142]:
response = model.generate_content([tp, img],safety_settings=safety_settings_block_none)
to_markdown(response.text)

>  The model is a young woman with long, wavy brown hair. She is wearing a pink and white shalwar kameez. The kameez is long and loose-fitting, with a V-neck and short sleeves. The shalwar are fitted and slightly flared at the bottom. The model is sitting on a stone wall in front of a stone building. The background is a blur of green and brown. The model is smiling and has her eyes closed. She is wearing a few pieces of jewelry, including a necklace, a bracelet, and earrings.

In [143]:
prompt_sdxl = "a photo of ahana an Indian middle class girl generate a realistic photo with beautiful eyes and" + response.text

In [144]:
prompt_sdxl

'a photo of ahana an Indian middle class girl generate a realistic photo with beautiful eyes and The model is a young woman with long, wavy brown hair. She is wearing a pink and white shalwar kameez. The kameez is long and loose-fitting, with a V-neck and short sleeves. The shalwar are fitted and slightly flared at the bottom. The model is sitting on a stone wall in front of a stone building. The background is a blur of green and brown. The model is smiling and has her eyes closed. She is wearing a few pieces of jewelry, including a necklace, a bracelet, and earrings.'

In [151]:
neg_prompt = """ unrealistic anatomy, blurry, high quality, clear face, unrealistic eyes, good lighting, unrealistic hair,
deformed, disfigured, extra limbs, missing limbs, fused limbs, bad anatomy, bad proportions,
poorly drawn hands, extra fingers, long neck, morbid, mutilated, unrealistic """

In [152]:
# image = open("ph.jpg", "rb")

In [153]:
%%time

input = {
    "width": 768,
    "height": 768,
    "prompt": prompt_sdxl,
    # "image": image,
    "refine": "expert_ensemble_refiner",
    "scheduler":"K_EULER_ANCESTRAL",
    "apply_watermark": False,
    "num_inference_steps": 200,
    "lora_scale":0.7,
    "num_outputs":1,
    "refine_steps":150,
    "guidance_scale":7.5,
    "high_noise_frac":0.8,
    "negative_prompt":neg_prompt,
    "prompt_strength":0.85,
    "disable_safety_checker":True,
}

output = replicate_client.run(
    "code8soumya/ai-model-ahana:5ef3bdff062556b09072edabe946bc497a6621436df5cf1858247d3b3d900ee8",
    input=input
)
print(output)

['https://replicate.delivery/pbxt/5f6lZu5ytww7bK0ywbeWCO2DFr6HaoEXQzYlnwifs3jv6Y2lA/out-0.png']
CPU times: user 337 ms, sys: 28.5 ms, total: 365 ms
Wall time: 35.1 s


In [ ]:
# neg_prompt = """ Avoid nudity, explicit sexual content, extreme poses, vulgar expressions,
# inappropriate gestures, overly suggestive clothing, and any form of explicit adult content. """

In [ ]:
# tp1 = """ Generate a detailed context from the image in strictly less than 60 words.
# Focus on the model's pose, body language, and clothing.
# Highlight how the model's posture and facial expression convey confidence and allure.
# Describe the key elements of the model's outfit,
# emphasizing how it enhances their figure and contributes to the overall stylish and modern atmosphere of the image. """